## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import sys
import os

# Import API key
sys.path.append('../')
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Date,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description
0,0,Lapeer,US,43.0514,-83.3188,2021-11-15 19:18:46,41.23,60,90,12.66,overcast clouds
1,1,Albany,US,42.6001,-73.9662,2021-11-15 19:06:11,45.18,79,100,1.01,overcast clouds
2,2,Punta Arenas,CL,-53.1500,-70.9167,2021-11-15 19:10:46,51.91,37,20,26.46,few clouds
3,3,Atuona,PF,-9.8000,-139.0333,2021-11-15 19:10:37,77.63,75,6,11.30,light rain
4,4,Saint Anthony,US,45.0205,-93.2180,2021-11-15 19:16:20,35.89,71,90,1.99,light rain


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 50
What is the maximum temperature you would like for your trip? 85


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Date,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description
2,2,Punta Arenas,CL,-53.1500,-70.9167,2021-11-15 19:10:46,51.91,37,20,26.46,few clouds
3,3,Atuona,PF,-9.8000,-139.0333,2021-11-15 19:10:37,77.63,75,6,11.30,light rain
5,5,Makakilo City,US,21.3469,-158.0858,2021-11-15 19:18:47,79.12,68,90,4.61,light rain
9,9,Hermanus,ZA,-34.4187,19.2345,2021-11-15 19:10:41,64.90,88,100,8.90,overcast clouds
10,10,Rio Grande,BR,-32.0350,-52.0986,2021-11-15 19:18:48,84.31,76,80,14.70,broken clouds
13,13,Bambous Virieux,MU,-20.3428,57.7575,2021-11-15 19:11:05,75.45,83,40,2.30,scattered clouds
14,14,Victoria,HK,22.2855,114.1577,2021-11-15 19:18:49,71.33,79,21,3.00,few clouds
16,16,Grand-Santi,GF,4.2500,-54.3833,2021-11-15 19:18:50,80.02,83,87,3.36,overcast clouds
23,23,Sur,OM,22.5667,59.5289,2021-11-15 19:18:53,75.54,49,54,3.69,broken clouds
26,26,Sechura,PE,-5.5569,-80.8222,2021-11-15 19:13:46,75.36,56,23,15.77,few clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                423
City                   423
Country                419
Lat                    423
Lng                    423
Date                   423
Max Temp               423
Humidity               423
Cloudiness             423
Wind Speed             423
Weather Description    423
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df.dropna()

,City_ID,City,Country,Lat,Lng,Date,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description
2,2,Punta Arenas,CL,-53.1500,-70.9167,2021-11-15 19:10:46,51.91,37,20,26.46,few clouds
3,3,Atuona,PF,-9.8000,-139.0333,2021-11-15 19:10:37,77.63,75,6,11.30,light rain
5,5,Makakilo City,US,21.3469,-158.0858,2021-11-15 19:18:47,79.12,68,90,4.61,light rain
9,9,Hermanus,ZA,-34.4187,19.2345,2021-11-15 19:10:41,64.90,88,100,8.90,overcast clouds
10,10,Rio Grande,BR,-32.0350,-52.0986,2021-11-15 19:18:48,84.31,76,80,14.70,broken clouds
...,...,...,...,...,...,...,...,...,...,...,...
695,695,Belmonte,BR,-15.8631,-38.8828,2021-11-15 19:21:49,72.10,90,100,13.04,overcast clouds
697,697,Englewood,US,39.6478,-104.9878,2021-11-15 19:21:50,76.32,15,75,1.01,broken clouds
699,699,Cukai,MY,4.2500,103.4167,2021-11-15 19:21:50,78.94,90,100,2.68,overcast clouds
701,701,Half Moon Bay,US,37.4636,-122.4286,2021-11-15 19:21:51,73.89,83,1,0.00,haze


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Weather Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
2,Punta Arenas,CL,51.91,few clouds,-53.1500,-70.9167,
3,Atuona,PF,77.63,light rain,-9.8000,-139.0333,
5,Makakilo City,US,79.12,light rain,21.3469,-158.0858,
9,Hermanus,ZA,64.90,overcast clouds,-34.4187,19.2345,
10,Rio Grande,BR,84.31,broken clouds,-32.0350,-52.0986,
13,Bambous Virieux,MU,75.45,scattered clouds,-20.3428,57.7575,
14,Victoria,HK,71.33,few clouds,22.2855,114.1577,
16,Grand-Santi,GF,80.02,overcast clouds,4.2500,-54.3833,
23,Sur,OM,75.54,broken clouds,22.5667,59.5289,
26,Sechura,PE,75.36,few clouds,-5.5569,-80.8222,


In [13]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"

    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print(f"If Hotel not found, skip the city..., index:{index}")
        

If Hotel not found, skip the city..., index:2
If Hotel not found, skip the city..., index:3
If Hotel not found, skip the city..., index:5
If Hotel not found, skip the city..., index:9
If Hotel not found, skip the city..., index:10
If Hotel not found, skip the city..., index:13
If Hotel not found, skip the city..., index:14
If Hotel not found, skip the city..., index:16
If Hotel not found, skip the city..., index:23
If Hotel not found, skip the city..., index:26
If Hotel not found, skip the city..., index:27
If Hotel not found, skip the city..., index:28
If Hotel not found, skip the city..., index:29
If Hotel not found, skip the city..., index:30
If Hotel not found, skip the city..., index:34
If Hotel not found, skip the city..., index:36
If Hotel not found, skip the city..., index:37
If Hotel not found, skip the city..., index:40
If Hotel not found, skip the city..., index:43
If Hotel not found, skip the city..., index:44
If Hotel not found, skip the city..., index:48
If Hotel not foun

If Hotel not found, skip the city..., index:300
If Hotel not found, skip the city..., index:302
If Hotel not found, skip the city..., index:304
If Hotel not found, skip the city..., index:305
If Hotel not found, skip the city..., index:306
If Hotel not found, skip the city..., index:307
If Hotel not found, skip the city..., index:313
If Hotel not found, skip the city..., index:314
If Hotel not found, skip the city..., index:315
If Hotel not found, skip the city..., index:316
If Hotel not found, skip the city..., index:317
If Hotel not found, skip the city..., index:318
If Hotel not found, skip the city..., index:321
If Hotel not found, skip the city..., index:322
If Hotel not found, skip the city..., index:323
If Hotel not found, skip the city..., index:324
If Hotel not found, skip the city..., index:326
If Hotel not found, skip the city..., index:327
If Hotel not found, skip the city..., index:330
If Hotel not found, skip the city..., index:334
If Hotel not found, skip the city..., in

If Hotel not found, skip the city..., index:577
If Hotel not found, skip the city..., index:578
If Hotel not found, skip the city..., index:579
If Hotel not found, skip the city..., index:580
If Hotel not found, skip the city..., index:581
If Hotel not found, skip the city..., index:582
If Hotel not found, skip the city..., index:584
If Hotel not found, skip the city..., index:588
If Hotel not found, skip the city..., index:589
If Hotel not found, skip the city..., index:591
If Hotel not found, skip the city..., index:592
If Hotel not found, skip the city..., index:599
If Hotel not found, skip the city..., index:600
If Hotel not found, skip the city..., index:602
If Hotel not found, skip the city..., index:606
If Hotel not found, skip the city..., index:607
If Hotel not found, skip the city..., index:609
If Hotel not found, skip the city..., index:611
If Hotel not found, skip the city..., index:613
If Hotel not found, skip the city..., index:614
If Hotel not found, skip the city..., in

In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head()

,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
2,Punta Arenas,CL,51.91,few clouds,-53.1500,-70.9167,
3,Atuona,PF,77.63,light rain,-9.8000,-139.0333,
5,Makakilo City,US,79.12,light rain,21.3469,-158.0858,
9,Hermanus,ZA,64.90,overcast clouds,-34.4187,19.2345,
10,Rio Grande,BR,84.31,broken clouds,-32.0350,-52.0986,


In [10]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Weather Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30, 30), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))